
## Projeto ICMC Júnior
### Objetivo: Prever se um funcionário vai sair da empresa (attrition).

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [41]:
# Leitura do arquivo csv
df = pd.read_csv('dados.csv')
# Verificação do tamanho da base de dados
df.shape

(1470, 35)

In [43]:
def calculate_lim_iqr(data_column):
    """Função responsável por calcular a amplitude interquartil de um Série (Coluna)"""
    Q1 = data_column.quantile(0.25) # 1° Quartil 
    Q3 = data_column.quantile(0.75) # 3° Quartil 
    
    
    IQR = Q3 - Q1 # amplitude interquartil
    
    limite_inferior = Q1 - (1.5 * IQR) # lim inferior do box-plot
    limite_superior = Q3 + (1.5 * IQR) # lim superior do box-plot
    
    return limite_inferior, limite_superior

def verify_outliers(column, print_dados = True):
    # Calcula limite inferior e superior da Série fornecida (coluna)
    lim_inf, lim_sup = calculate_lim_iqr(df[column])
    
    # Verifica se possui um dado menor que o limite inferior ou se há um dado maior que o limite superior
    if df[column].min() < lim_inf or df[column].max() > lim_sup:
        if print_dados:
            print(f"{column}: lim_inf: {lim_inf} | lim_sup:{lim_sup} | min:{df[column].min()} | max:{df[column].max()}")
        return True
    return False

In [44]:
# Exibe os 5 primeiros funcionários da base de dados, apenas para contextualizar com a base
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,2,Life Sciences,1.0,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,1,Life Sciences,1.0,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2.0,2,Other,1.0,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,NaN,4,Life Sciences,NaN,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,NaN,591,Research & Development,2.0,1,Medical,NaN,7,...,4,80,1,6,3,3,2,2,2,2


In [45]:
# TRATAMENTO DE VALORES NULOS
df.isnull().sum() # Verifica quantos valores nulos tem por colunas, ou seja, por categoria.

Age                           0
Attrition                     0
BusinessTravel               70
DailyRate                     0
Department                    0
DistanceFromHome             47
Education                     0
EducationField                0
EmployeeCount               220
EmployeeNumber                0
EnvironmentSatisfaction      53
Gender                       67
HourlyRate                   32
JobInvolvement               61
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                     51
PercentSalaryHike             0
PerformanceRating            46
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [46]:
# Tratamento dos valores faltantes - EmployeeCount
print(df['EmployeeCount'].describe())

count    1250.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: EmployeeCount, dtype: float64


In [48]:
# Em 'EmployeeCount' todos os valores são como padrão igual a 1.0, sendo cada linha, um funcionário, com isso se corrige os NaN para 1.0
# no dataframe original
df['EmployeeCount'] = df['EmployeeCount'].fillna(value=1)

In [49]:
# Tratamento dos valores faltantes - DistanceFromHome (Quantitativa continua)
print(df['DistanceFromHome'].describe())

# Calcula o limite inferior e limite superior da coluna DistanceFromHome
lim_inf_dfh,lim_sup_dfh = calculate_lim_iqr(df['DistanceFromHome'])

print(f"Limite Inferior (Q1 - 1.5*IQR): {lim_inf_dfh}")
print(f"Limite Superior (Q3 + 1.5*IQR): {lim_sup_dfh}")

count    1423.000000
mean        9.188335
std         8.133707
min         1.000000
25%         2.000000
50%         7.000000
75%        14.000000
max        29.000000
Name: DistanceFromHome, dtype: float64
Limite Inferior (Q1 - 1.5*IQR): -16.0
Limite Superior (Q3 + 1.5*IQR): 32.0


In [50]:
# O menor valor (min) é igual a 1.0 e o maior valor (max) é igual a 29.0. Com isso, podemos ver que todos 
# os dados então dentro das margens, não tendo nenhum outliers. No entanto, devido ao desvio padrão ser mais de 80% da média, 
# é visível que os dados não estão bem distribuidos. Para isso, opta-se por preencher os valores nulos com a mediana da coluna.

# Calculo a mediana
median_distances = df['DistanceFromHome'].median() 
# Preenche todos os valores NaN com a mediana e em seguida, reatribui a coluna nela mesma.
df['DistanceFromHome'] = df['DistanceFromHome'].fillna(median_distances)


In [51]:
# Tratamento da coluna EnvironmentSatisfaction - Qualitativa ordinal
print(df['EnvironmentSatisfaction'].value_counts(dropna=False))

# Preenche os valores ausentes com os valores da moda, em seguida, reatribui os valores na coluna original do dataframe
# Obs: a mediana aqui teria o mesmo resultado da moda.

# Pega o primeiro (e único) valor da moda
moda_env_satisfaction = df['EnvironmentSatisfaction'].mode()[0]
df['EnvironmentSatisfaction'] = df['EnvironmentSatisfaction'].fillna(moda_env_satisfaction)

EnvironmentSatisfaction
3.0    438
4.0    427
2.0    277
1.0    275
NaN     53
Name: count, dtype: int64


In [52]:
# Tratamento de valores ausentes coluna - OverTime (Qualitativo nominal)
print(df['OverTime'].value_counts(dropna=False, normalize=True)) # Exibe a proporção de cada resposta na coluna OverTime, variando de 0 a 1.

# Como podemos ver com o describe, há uma quantidade esmagadora de funcionários que não realizam horas extras, quase 70% deles.
# Preenche os valores ausentes com a moda e reatribui os resultados na coluna respectiva
df['OverTime'] = df['OverTime'].fillna(df['OverTime'].mode()[0])


OverTime
No     0.693197
Yes    0.272109
NaN    0.034694
Name: proportion, dtype: float64


In [53]:
# Tratamento valores ausentes - HourlyRate (Quantitativo Contínua)
print(df['HourlyRate'].describe())

# Calcula o limite inferior e limite superior da coluna HourlyRate
lim_inf_hr, lim_sup_hr = calculate_lim_iqr(df['HourlyRate']) 
print(f"Limite inferior:{lim_inf_hr}, limite superior: {lim_sup_hr}")
print(f"Coeficiente de variação: {df['HourlyRate'].std() / df['HourlyRate'].mean()}")
print(f"Média: { df['HourlyRate'].mean()}, Mediana: { df['HourlyRate'].median()} e Moda: { df['HourlyRate'].mode()[0]}")

count    1438.000000
mean       65.907510
std        20.386193
min        30.000000
25%        48.000000
50%        66.000000
75%        84.000000
max       100.000000
Name: HourlyRate, dtype: float64
Limite inferior:-6.0, limite superior: 138.0
Coeficiente de variação: 0.3093151741293963
Média: 65.90751043115438, Mediana: 66.0 e Moda: 66.0


In [54]:
# Podemos ver atráves dos limites (inferior e superior) que os nossos dados estão dentro dessas margens, visto que
# o menor valor (min) é 30.0 e o maior (max) é 100.0, com isso não possui outliers. O coeficiente de variação é igual a 30%, dessa forma
# temos uma variação média entre os dados. Além disso, a média está bem proxima da mediana. Usa-se a média para representação dos dados ausentes

# Preenche os valores ausentes com a média e reatribui os resultados na coluna respectiva
df['HourlyRate'] = df['HourlyRate'].fillna(df['HourlyRate'].mean())

In [55]:
# Tratamento de valores ausentes - BusinessTravel (Qualitativo nominal)
print(df['BusinessTravel'].describe())

count              1400
unique                3
top       Travel_Rarely
freq                989
Name: BusinessTravel, dtype: object


In [56]:
# Dada a predominância (989 em 1400) de "Travel_Rarely" e o número de valores nulos não tão expressivo,
# é válido substituir os valores ausentes por essa pela moda
df['BusinessTravel'] = df['BusinessTravel'].fillna('Travel_Rarely')

In [57]:
# Tratamento de valores ausentes - Gender (Qualitativo nominal)
print(df['Gender'].describe())

count     1403
unique       2
top       Male
freq       846
Name: Gender, dtype: object


In [58]:
# Apesar de a distribuição ser relativamente equilibrada, dado que o número de valores nulos não é tão expressivo
# e que estamos tratando um fator qualitativo nominal, opta-se por substituir os valores ausentes pela moda
df['Gender'] = df['Gender'].fillna('Male')

In [59]:
# Tratamento de valores ausentes - JobInvolvement (Qualitativo ordinal)
print(df['JobInvolvement'].describe())

# Calcula-se os limites inferior e superior da coluna JobInvolvement
lim_inf_hr, lim_sup_hr = calculate_lim_iqr(df['JobInvolvement']) 
print(f"Limite inferior: {lim_inf_hr}, limite superior: {lim_sup_hr}")
print(f"Coeficiente de variação: {df['JobInvolvement'].std() / df['JobInvolvement'].mean()}")
print(f"Média: { df['JobInvolvement'].mean()}, Mediana: { df['JobInvolvement'].median()} e Moda: { df['JobInvolvement'].mode()[0]}")

count    1409.000000
mean        2.732434
std         0.707171
min         1.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         4.000000
Name: JobInvolvement, dtype: float64
Limite inferior: 0.5, limite superior: 4.5
Coeficiente de variação: 0.2588061942246768
Média: 2.7324343506032647, Mediana: 3.0 e Moda: 3.0


In [60]:
# Não há outliers na coluna e o coeficiente de variação é baixo, indicando que os dados estão próximos da média.
# Entretanto, como a média está num valor decimal, é próxima da mediana e a moda tem mesmo valor da mediana,
# opta-se por preencher os valores ausentes com a mediana
df['JobInvolvement'] = df['JobInvolvement'].fillna(df['JobInvolvement'].median())

In [62]:
# Tratamento de valores ausentes - PerformanceRating (Qualitativo ordinal)
print(df['PerformanceRating'].describe())

# Calcula-se os limites inferior e superior da coluna PerformanceRating
lim_inf_hr, lim_sup_hr = calculate_lim_iqr(df['PerformanceRating']) 
print(f"Limite inferior: {lim_inf_hr}, limite superior: {lim_sup_hr}")
print(f"Coeficiente de variação: {df['PerformanceRating'].std() / df['PerformanceRating'].mean()}")
print(f"Média: { df['PerformanceRating'].mean()}, Mediana: { df['PerformanceRating'].median()} e Moda: { df['PerformanceRating'].mode()[0]}")

count    1424.000000
mean        3.150281
std         0.357472
min         3.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         4.000000
Name: PerformanceRating, dtype: float64
Limite inferior: 3.0, limite superior: 3.0
Coeficiente de variação: 0.11347308107531523
Média: 3.1502808988764044, Mediana: 3.0 e Moda: 3.0


In [63]:
# Dada a vasta maioria dos funcionários possuírem PerformanceRating igual a 3, é válido substituir os valores ausentes pela mediana/moda
df['PerformanceRating'] = df['PerformanceRating'].fillna(df['PerformanceRating'].mode()[0])

In [67]:
# Após isso, verifica-se se ainda há valores ausentes na base de dados
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [77]:
# Verifica se possui funcionários duplicados
df.duplicated().sum() 

np.int64(0)

In [68]:
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.118367,2.912925,1.0,1024.865306,2.727891,65.907510,2.743537,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.011794,1.024165,0.0,602.024335,1.074222,20.162929,0.694387,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,65.907510,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,13.000000,4.000000,1.0,1555.750000,4.000000,83.000000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [69]:
# !adicionar depois as variávies que possuem valores ausentes ainda 
# ('BusinessTravel','Gender', 'JobInvolvement', 'PerformanceRating') -- Cols_qualitativas

# (Texto ou Números que são Rótulos/Categorias) 
cols_qualitativas = [
    'Attrition', 
    'Department', 
    'Education',
    'EducationField',
    'EnvironmentSatisfaction',
    'JobLevel', 
    'JobRole', 
    'JobSatisfaction',
    'MaritalStatus',
    'OverTime', 
    'RelationshipSatisfaction',
    'StockOptionLevel',
    'WorkLifeBalance'
]

# (Números que são Medidas/Contagens)
cols_quantitativas = [
    'Age',
    'DailyRate',
    'DistanceFromHome',
    'HourlyRate',
    'MonthlyIncome',
    'MonthlyRate',
    'NumCompaniesWorked',
    'PercentSalaryHike',
    'TotalWorkingYears',
    'TrainingTimesLastYear',
    'YearsAtCompany',
    'YearsInCurrentRole',
    'YearsSinceLastPromotion',
    'YearsWithCurrManager'
]

# Add após o tratamento dos valores nulos 
#! cols_qualitativas.extend(['BusinessTravel','Gender', 'JobInvolvement', 'PerformanceRating'])


# Verificação se alguma das colunas quantitativa possui outliers 
print(10*'-','Colunas que podem conter outliers:', 10*'-')
for i in cols_quantitativas:
    verify_outliers(i)

print(f"\n{10*'-'}Colunas que pode conter dados inconsistentes{10*'-'}")
for coluna in cols_qualitativas:
    print(f"\n===== Análise da Coluna: {coluna} =====")
    print(df[coluna].value_counts())
    print("---------------------------------")        
    

---------- Colunas que podem conter outliers: ----------
MonthlyIncome: lim_inf: -5291.0 | lim_sup:16581.0 | min:1009 | max:19999
NumCompaniesWorked: lim_inf: -3.5 | lim_sup:8.5 | min:0 | max:9
TotalWorkingYears: lim_inf: -7.5 | lim_sup:28.5 | min:0 | max:40
TrainingTimesLastYear: lim_inf: 0.5 | lim_sup:4.5 | min:0 | max:6
YearsAtCompany: lim_inf: -6.0 | lim_sup:18.0 | min:0 | max:40
YearsInCurrentRole: lim_inf: -5.5 | lim_sup:14.5 | min:0 | max:18
YearsSinceLastPromotion: lim_inf: -4.5 | lim_sup:7.5 | min:0 | max:15
YearsWithCurrManager: lim_inf: -5.5 | lim_sup:14.5 | min:0 | max:17

----------Colunas que pode conter dados inconsistentes----------

===== Análise da Coluna: Attrition =====
Attrition
No     1233
Yes     237
Name: count, dtype: int64
---------------------------------

===== Análise da Coluna: Department =====
Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64
---------------------------------

==

In [80]:
# Podemos que será necessário tratar os outliers de algumas colunas.
# Da saída acima é possível verificar também, que não possuimos valores inconsistes para erros de digitação, mas podemos verificar alguns
# valores que podem estar como algum valor do HourlyRate > DailyRate

print((df['HourlyRate'] > df['DailyRate']).sum()) # Cria uma Série booleana e soma todos os casos em que a condição é True
print((df['DailyRate'] > df['MonthlyRate']).sum()) # Cria uma Série booleana e soma todos os casos em que a condição é True

0
0


In [81]:
#  Tratamento de outliers

df[['MonthlyIncome',
'NumCompaniesWorked',
'TotalWorkingYears',
'TrainingTimesLastYear',
'YearsAtCompany',
'YearsInCurrentRole',
'YearsSinceLastPromotion',
'YearsWithCurrManager',]].describe()

,MonthlyIncome,NumCompaniesWorked,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,6502.931293,2.693197,11.279592,2.799320,7.008163,4.229252,2.187755,4.123129
std,4707.956783,2.498009,7.780782,1.289271,6.126525,3.623137,3.222430,3.568136
min,1009.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2911.000000,1.000000,6.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,4919.000000,2.000000,10.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,8379.000000,4.000000,15.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,19999.000000,9.000000,40.000000,6.000000,40.000000,18.000000,15.000000,17.000000


In [82]:
# Utilizar mediana ou tranformar os dados utilizando log (os dados apresentados nos outliers não são erros, são dados reais)
# transformar algumas colunas, visto que nosso objetivo é um problema de ML de classificação binária, ou seja, 
# se um funcionário vai sair ou não da empresa (0 - no, 1 - yes)

In [83]:
# Tratando os outliers para tratar a cauda longa da distribuição, utilizando Log Transform
# Isso é feito para deixar os dados mais simétricos, e deixa a média e o desvio padrão mais representativos
# Aplicando a transformação log nas colunas com assimetria mais óbvias 

# np.log1p(x) é o mesmo que log(x+1), para evitar erro de log(0)
df['MonthlyIncome_log'] = np.log1p(df['MonthlyIncome'])
df['TotalWorkingYears'] = np.log1p(df['TotalWorkingYears'])
df['YearsAtCompany'] = np.log1p(df['YearsAtCompany'])
df['YearsInCurrentRole'] = np.log1p(df['YearsInCurrentRole'])
df['YearsSinceLastPromotion'] = np.log1p(df['YearsSinceLastPromotion'])




In [84]:
# Mapeamento do nosso atributo alvo utilizando LabelEncoder para converter do tipo nominal para o tipo numérico
encoder = LabelEncoder()
df['Attrition'] = encoder.fit_transform(df['Attrition'])  # Converter do tipo nominal 'Yes' or 'No' para 1 ou 0 
# O motivo dessa modificação se dá porque quando formos aplicar a modelagem o algoritmo de ML irá retornar um probabilidade entre 0 e 1 
# Se o funcionário vai sair ou não da empresa

In [85]:
# Realizando o mesmo mapeamento para os atributos 'OverTime' e 'Gender' só que agora utilizando o map do pandas
df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})
# df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0}) Realizar após o tratamento de valores nulos